# Nettoyage des données

Après avoir scrappé, il nous faut tenir compte de plusieurs choses pour nettoyer les bases de données et les fusionner :
- dans un premier temps, on convertit les dates dans un format date permettant ensuite de trier et de réarranger la table dans le bon ordre temporel.
- les cases vides susceptibles d'apparaître dans les colonnes "Likes", "Views", etc... sont remplacées par des 0 (il n'y a eu en effet aucun like par exemple).
- on fusionne les tables et on les trie par date
- enfin, on supprime les bots, définis dans notre code par une répétition de 4 tweets faisant moins de 1000 vues (car certains tweets qui se répètent peuvent être des titres d'article)


In [ ]:
!pip install pandas 
!pip install datetime

## 1. Convertir les dates en format date

Dans le code suivant, nous cherchons d'une part à remplacer les 2.3K par 2300, à convertir les dates dans un format *datetime*.

Nous avons fait le choix d'ajouter deux colonnes :
- une colonne YearWeek sous format 2024-1 pour la première semaine de 2024
- une colonne YearMonth sous format 2024-10 pour le mois d'octobre 2024

Nous avons décidé de nous restreindre à l'année 2024, en excluant le mois de décembre (car pas complet) : en effet, les fluctuations de 2023 étaient relativement similaires à celles de début de début 2024.

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# Fonction 1 : Nettoyer les données numériques
def convertir_en_nombre(valeur):
    """
    Convertit les chaînes de caractères (ex: '1.2K') en nombres.
    Remplace NaN par 0.
    """
    if pd.isna(valeur):  # Si la valeur est NaN, retourne 0
        return 0
    if isinstance(valeur, str):  # Vérifie si la valeur est une chaîne de caractères
        valeur = valeur.strip()  # Supprime les espaces inutiles
        if 'K' in valeur:  # Si la valeur contient 'K', multiplier par 1 000
            return float(valeur.replace('K', '')) * 1000
        elif 'M' in valeur:  # Si la valeur contient 'M', multiplier par 1 000 000
            return float(valeur.replace('M', '')) * 1000000
    try:
        return float(valeur)  # Sinon, convertir directement en nombre
    except ValueError:
        return 0  # Si la conversion échoue, retourner 0

# Fonction 2 : Conversion des dates
def convert_date(value, previous_date):
    """
    Convertit les valeurs de date relative ou absolue en objets datetime.
    """
    try:
        value = str(value).strip()  # S'assurer que la valeur est une chaîne
        
        # Cas 1 : Format "Nov 5" (pas d'année explicite)
        if len(value.split()) == 2:
            value_with_year = value + " 2024"  # Ajouter l'année par défaut
            return datetime.strptime(value_with_year, '%b %d %Y')
        
        # Cas 2 : Format complet "Nov 5, 2023"
        elif len(value.split()) == 3:
            return datetime.strptime(value, '%b %d, %Y')
        
        # Cas 3 : Valeur non reconnue
        else:
            return pd.NaT
    except Exception as e:
        print(f"Erreur lors de la conversion de la date : {value} -> {e}")
        return pd.NaT  # Retourne NaT si la conversion échoue

# Fonction 3 : Nettoyer et transformer un fichier
def process_file(file_path):
    """
    Nettoie et transforme un fichier Excel, en ajoutant des colonnes converties
    et en générant deux fichiers de sortie :
    1. Toutes les données transformées.
    2. Données filtrées avant une date spécifique.
    """
    print(f"Traitement du fichier : {file_path}")
    # Charger les données
    df = pd.read_excel(file_path)
    
    # Nettoyer les colonnes numériques
    colonnes_a_nettoyer = ['Comments', 'Repost', 'Likes', 'Views']
    for colonne in colonnes_a_nettoyer:
        if colonne in df.columns:
            df[colonne] = df[colonne].apply(convertir_en_nombre)
    
    # Convertir les dates
    converted_dates = []
    previous_date = None
    for value in df['Date']:
        if previous_date is None:  # La première ligne doit être une date absolue
            converted_date = convert_date(value, datetime(2024, 1, 1))  # Supposition du début
        else:
            converted_date = convert_date(value, previous_date)
        converted_dates.append(converted_date)
        if pd.notna(converted_date):  # Mettre à jour la référence seulement si la conversion a réussi
            previous_date = converted_date
    
    # Ajouter les colonnes converties
    df['ConvertedDate'] = converted_dates
    df['YearWeek'] = df['ConvertedDate'].dt.strftime('%Y-%U')
    df['YearMonth'] = df['ConvertedDate'].dt.strftime('%Y-%m')
    


    # Filtrer les données pour garder uniquement les dates entre le 1er janv et le 1er décembre 2024
    # Définir les bornes de la période
    start_date = datetime(2024, 1, 1)  # 1er janvier 2024 inclus
    end_date = datetime(2024, 11, 30)  # 30 novembre 2024 inclus

    # Appliquer le filtre pour garder les dates entre ces bornes
    df_filtered = df[(df['ConvertedDate'] >= start_date) & (df['ConvertedDate'] <= end_date)]

    
    # Sauvegarder les données filtrées : on renomme les fichiers en ajoutant _date à la fin
    output_file_filtered = file_path.replace('.xlsx', '_date.xlsx')
    df_filtered.to_excel(output_file_filtered, index=False)


# Liste des fichiers Excel à traiter
files = ['/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_stay_1.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_exode_1.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_exode_2.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_2.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_3.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_4.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_5.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_6.xlsx', 
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_leave_p1.xlsx', 
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_quit.xlsx']

# Boucle principale pour traiter chaque fichier
for file in files:
    process_file(file)



Traitement du fichier : /home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_stay_1.xlsx


FileNotFoundError: [Errno 2] No such file or directory: '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_stay_1.xlsx'

## 2. Fusion des tables et tri par date

On fusionne les tables et on trie par date : on arrive au fichier tweets_fusionnes.xlsx.

In [21]:
import pandas as pd

file_list = [
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_stay_1_date.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_exode_1_date.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_exode_2_date.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_2_date.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_3_date.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_4_date.xlsx',
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_5_date.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_6_date.xlsx', 
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_last_date.xlsx', '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_leave_p1_date.xlsx', 
'/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_scrap/tweets_quit_date.xlsx'
   
]

# Charger et fusionner tous les fichiers
dataframes = []  # Liste pour stocker les DataFrames
for file in file_list:
    df = pd.read_excel(file)  # Charger chaque fichier
    dataframes.append(df)  # Ajouter le DataFrame à la liste

# Fusionner tous les DataFrames en un seul
merged_df = pd.concat(dataframes, ignore_index=True)





## 3. Suppression des bots

Nous nous sommes rendus compte que certains tweets revenaient plusieurs fois, après avoir réfléchi et testé plusieurs façons de supprimer les bots, nous nous sommes arrêtés à la définition d'un bot comme un tweet identique qui revient 4 fois ou plus, et qui fait moins de 1000 vues. 

In [22]:
df = merged_df

# Vérifier que les colonnes nécessaires existent
if 'Content' not in df.columns or 'Views' not in df.columns:
    raise ValueError("Les colonnes 'Content' et 'Views' sont nécessaires pour ce traitement.")

# Compter les occurrences de chaque tweet
content_counts = df['Content'].value_counts()

# Filtrer les contenus apparaissant au moins 4 fois
repeated_contents = content_counts[content_counts >= 4].index

# Identifier les tweets répétitifs
repeated_tweets = df[df['Content'].isin(repeated_contents)]

# Identifier les groupes où un tweet a plus de 1000 vues
to_keep = repeated_tweets.groupby('Content')['Views'].max()  # Max des vues pour chaque groupe
valid_tweets = to_keep[to_keep > 1000].index  # Conserver les groupes avec au moins un tweet > 1000 vues

# Séparer les tweets répétitifs en deux groupes
# Tweets à conserver (au moins 1 tweet > 1000 vues dans le groupe)
tweets_to_keep = repeated_tweets[repeated_tweets['Content'].isin(valid_tweets)]

# Tweets à supprimer (aucun tweet > 1000 vues dans le groupe)
tweets_to_remove = repeated_tweets[~repeated_tweets['Content'].isin(valid_tweets)]

# Combiner les données finales
df_cleaned = pd.concat([df[~df['Content'].isin(repeated_contents)], tweets_to_keep])

# Sauvegarder les tweets répétitifs supprimés
# tweets_to_remove.to_excel('removed_repeated_tweets.xlsx', index=False)

# Sauvegarder la base nettoyée
df_cleaned.to_excel('/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_fusionnes.xlsx', index=False)

On trie toute la table nettoyée sans les bots par date pour avoir la base finale :

In [23]:
# Vérifier que la colonne 'ConvertedDate' est bien en format datetime
if not pd.api.types.is_datetime64_any_dtype(merged_df['ConvertedDate']):
    merged_df['ConvertedDate'] = pd.to_datetime(merged_df['ConvertedDate'])

# Trier par date décroissante
merged_df = merged_df.sort_values(by='ConvertedDate', ascending=False)

# Sauvegarder la table fusionnée et triée
output_file = '/home/onyxia/work/Scrapping_tweets/Scrapping_tweets/tweets_fusionnes.xlsx'
merged_df.to_excel(output_file, index=False)

## 4. Jointure avec les résultats du NLP

On a une colonne en plus avec :
- 0 : le tweet n'est pas violent
- 1 : le tweet est violent 

In [ ]:
import csv
import pandas as pd

In [4]:
read_file = pd.read_excel("tweets_fusionnes.xlsx")
read_file.to_csv("tweets_fusionnes.csv", index = None, header = True) #conversion Excel en CSV

df_date = pd.read_csv("tweets_fusionnes.csv")
df_nlp = pd.read_csv("labeled_data_with_predictions.csv")

df_date = df_date.drop('Date', axis=1)

if len(df_date) != len(df_nlp):
    raise ValueError("Il n'y a pas le même nombre de lignes dans les deux dataframes")

df_fusion = pd.concat([df_date, df_nlp], axis=1)
df_fusion.to_csv('tables_fusion.csv', index = False)

print(df_fusion.head())

ValueError: Il n'y a pas le même nombre de lignes dans les deux dataframes